<a href="https://colab.research.google.com/github/vibhorrawal/code-backup/blob/master/saving_weights_in_a_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import keras
from keras import models
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import SGD
from keras import layers
from keras.layers import  Dense, Flatten, Conv2D, MaxPooling2D
from keras import Input

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!ls

drive  sample_data


In [6]:
!unzip 'drive/My Drive/Colab Notebooks/datasets/plant_village.zip'

Archive:  drive/My Drive/Colab Notebooks/datasets/plant_village.zip
   creating: plant_village/
   creating: plant_village/test/
   creating: plant_village/test/Apple_Frogeye_Spot/
  inflating: plant_village/test/Apple_Frogeye_Spot/4231e86c-5f4c-439c-be0f-1fa0274581c6___JR_FrgE.S 3067.JPG  
  inflating: plant_village/test/Apple_Frogeye_Spot/5435d1b3-2138-47ca-aca4-23cac9d1bb43___JR_FrgE.S 2779.JPG  
  inflating: plant_village/test/Apple_Frogeye_Spot/717c4f62-f05f-4345-a5d7-3b34f3a2efb6___JR_FrgE.S 2836.JPG  
  inflating: plant_village/test/Apple_Frogeye_Spot/723e8edc-3701-4ba1-bda2-7e0ebd535797___JR_FrgE.S 2837.JPG  
  inflating: plant_village/test/Apple_Frogeye_Spot/789c7185-b853-4638-b621-03ee71ce0c7e___JR_FrgE.S 8641.JPG  
  inflating: plant_village/test/Apple_Frogeye_Spot/804f69a4-cd0f-4579-9b9d-b1a032160642___JR_FrgE.S 3018.JPG  
  inflating: plant_village/test/Apple_Frogeye_Spot/8267ac2d-0e8f-4ed3-9da2-8180b99cc26a___JR_FrgE.S 8646.JPG  
  inflating: plant_village/test/Apple_Frog

In [7]:
# Normalize training and validation data in the range of 0 to 1
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Read the training sample and set the batch size 
train_generator = train_datagen.flow_from_directory(
        'plant_village/train/',
        target_size=(64, 64),
        batch_size=16,
        class_mode='categorical')

# Read Validation data from directory and define target size with batch size
validation_generator = validation_datagen.flow_from_directory(
        'plant_village/val/',
        target_size=(64, 64),
        batch_size=16,
        class_mode='categorical',
        shuffle=False)

test_generator = test_datagen.flow_from_directory(
        'plant_village/test/',
        target_size=(64, 64),
        batch_size=1,
        class_mode='categorical',
        shuffle=False)


Found 2217 images belonging to 4 classes.
Found 635 images belonging to 4 classes.
Found 319 images belonging to 4 classes.


In [0]:
from PIL import Image
import os, sys


dirs = os.listdir("plant_village/val/Apple_Frogeye_Spot/")
def resize():
    for item in dirs: 
        if os.path.isfile("plant_village/val/Apple_Frogeye_Spot/"+item):
            im = Image.open("plant_village/val/Apple_Frogeye_Spot/"+item)
            f, e = os.path.splitext("plant_village/val/Apple_Frogeye_Spot/"+item)
            imResize = im.resize((64, 64), Image.ANTIALIAS)
            imResize.save(f + '.jpg', 'JPEG', quality=90)

resize()

In [29]:
# Create the model
model = models.Sequential()
# Add new layers
model.add(Conv2D(128, kernel_size=(3,3), activation = 'relu', input_shape=(64,64,3)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 62, 62, 128)       3584      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 31, 31, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 29, 29, 64)        73792     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 12, 12, 64)        36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2304)              0         
__________

In [0]:
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['acc'])

In [31]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath="/content/drive/My Drive/Colab Notebooks/datasets/weights_try.hdf5", verbose=1, save_best_only=True)

history = model.fit_generator(train_generator, 
      steps_per_epoch=train_generator.samples/train_generator.batch_size,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1, callbacks=[checkpointer])

AttributeError: ignored

In [0]:
model = model.load_weights('/content/drive/My Drive/Colab Notebooks/datasets/weights_try.hdf5')

In [32]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 12)                108       
_________________________________________________________________
dense_7 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 9         
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________


In [20]:
model1 = models.Sequential()
model1.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='relu'))
model1.add(Dense(8, kernel_initializer='uniform', activation='relu'))
model1.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
# load weights
model1.load_weights('/tmp/weights.hdf5')
model1.summary()

ValueError: ignored